# Overview

Standard checks / results presentation.

It loads a parquet data frame that has been created in a different script.  You can also load that data into the viewer script, which has a search interface.

This is more about having a standard set of tables / charts that are top level checks to look through.

Originally it was at the end of the main script.

# Imports

In [27]:
# Safe imports
from datetime import datetime, timedelta, date

In [28]:
import pandas as pd
import numpy as np

In [29]:
import ipywidgets as widgets

In [30]:
# Imports on files that might have dependencies that need installing
import data_pier_querying
from athena_querying import AthenaQuery
from athena_common_queries import *
import user_agents # this converts user agent from browser to mobile / desktop etc.

# Settings & Loading Data

In [90]:
# load_merged_resutls_file_path = "merged_df_with_meta_01-10_01-12.gzip"
load_merged_results_file_path = "merged_df_with_meta_20200115_0000_to_20200122_0000.gzip"
just_sg_hk = True # a little bit dangerous as you'll also miss out ones without country set.

In [69]:
merged_df_with_meta = pd.read_parquet(load_merged_results_file_path)

In [92]:
if just_sg_hk:
    merged_df_with_meta = merged_df_with_meta[merged_df_with_meta.country_code.isin(["sg", "hk"])]

In [93]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code,user_agent,device_family,os_family,os_version,browser_family,browser_version,is_bot
index,,,,,,,,,,,,,,,,,,
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
2,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
3,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
4,00004699-493e-4238-a2ba-8cc38c829d45,PageView,https://www.moneysmart.hk/zh-hk/credit-cards,2020-01-21,1,1,shop,/zh-hk/credit-cards,/credit-cards,control,hk,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0


In [94]:
len(merged_df_with_meta)

1945676

In [95]:
merged_df_with_meta.dtypes

anonymous_id         object
event_name           object
page_url             object
date               category
s_count               int64
k_count               int64
page_type          category
slug               category
slug_root            object
ab_test            category
country_code       category
user_agent           object
device_family      category
os_family          category
os_version         category
browser_family     category
browser_version    category
is_bot              float16
dtype: object

# Common Code

In [96]:
def colour_grouped_table(df):

    def color_how_good(value):
        if isinstance(value, str):
            return
        av = abs(value)
        if av<2:
            c = "green"
        elif value <0:
            c =  "red"

        else:
            c= "blue"

        if av>20:
            return "background-color:rgb(250,200,200)"
        return "color:%s"%c # it's just CSS, so you can do background as well.
    return df.style.applymap(color_how_good , subset=["k_vs_s_%"])

In [97]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

# Top Level Checks

In [113]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

In [114]:

def highlight_cols(cell):
    #use hex colours, or named ones to ensure excel compatibility on export
    if cell=="":
        return ""
    ci = min(100, int(abs(cell*10)))
    if abs(cell)<=2:
        return "color:green;"
    if cell <0:

        return "background-color:#%02x%02x%02x;" % (255,255-ci,255-ci)
    if cell>0:
        #return "background-color:cyan;"
        return "background-color:#%02x%02x%02x;" % (255-ci, 255-ci, 255)

    return "background-color:red;"


def color_how_good(value): # old one, might be junk
    if isinstance(value, str):
        return
    av = abs(value)
    if av<2:
        c = "green"
    elif value <0:
        c =  "red"

    else:
        c= "blue"

    if av>20:
        return "background-color:rgb(250,200,200)"
    return "color:%s"%c # it's just CSS, so you can do background as well.

In [115]:
def colour_grouped_table(df):

    return df.style.applymap(highlight_cols, subset=["k_vs_s_%"])


In [116]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code,user_agent,device_family,os_family,os_version,browser_family,browser_version,is_bot
index,,,,,,,,,,,,,,,,,,
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
2,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
3,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
4,00004699-493e-4238-a2ba-8cc38c829d45,PageView,https://www.moneysmart.hk/zh-hk/credit-cards,2020-01-21,1,1,shop,/zh-hk/credit-cards,/credit-cards,control,hk,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0


## By Country


In [117]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "date"])

In [119]:
colour_grouped_table(g)

,country_code,date,s_count,k_count,is_bot,k_vs_s_%
7,hk,2020-01-15,83728,82846,179,-1.1
8,hk,2020-01-16,82069,81301,99,-0.9
9,hk,2020-01-17,76961,76521,265,-0.6
10,hk,2020-01-18,72513,71925,159,-0.8
11,hk,2020-01-19,70500,69852,155,-0.9
12,hk,2020-01-20,78667,77785,134,-1.1
13,hk,2020-01-21,71455,71478,95,0
28,sg,2020-01-15,350042,347661,413,-0.7
29,sg,2020-01-16,335199,332698,438,-0.7
30,sg,2020-01-17,329538,328105,407,-0.4


## By Event Type

In [120]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "event_name"])
g.sort_values(["country_code", "event_name"], inplace=True)
colour_grouped_table(g)

,country_code,event_name,s_count,k_count,is_bot,k_vs_s_%
35,hk,Display user feedback form,9,0,0,-100
37,hk,LeadGeneration.ClickConversion,638,677,0,6.1
38,hk,LeadGeneration.ClickedApplyButton,826,880,0,6.5
39,hk,LeadGeneration.ClickedCTA,2797,2929,0,4.7
41,hk,LeadGeneration.Conversion,5080,5023,0,-1.1
43,hk,LeadGeneration.FormSubmitted,975,1009,0,3.5
45,hk,LeadGeneration.RedirectCompleted,4685,4840,0,3.3
46,hk,LeadGeneration.ShowedMoreDetails,10947,11116,0,1.5
47,hk,LeadGeneration.ViewedMoreDetails,5665,5910,0,4.3
49,hk,PageView,225587,223683,1005,-0.8


In [121]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.event_name=="LeadGeneration.ClickedApplyButton"], ["country_code", "slug_root", "event_name"])

In [122]:
g

,country_code,slug_root,event_name,s_count,k_count,is_bot,k_vs_s_%
3,hk,/credit-cards,LeadGeneration.ClickedApplyButton,826.0,880.0,0.0,6.5
12,sg,/credit-cards,LeadGeneration.ClickedApplyButton,3594.0,3880.0,0.0,8.0
13,sg,/debt-consolidation-plan,LeadGeneration.ClickedApplyButton,164.0,172.0,0.0,4.9
14,sg,/personal-loan,LeadGeneration.ClickedApplyButton,989.0,1046.0,0.0,5.8


## By Top Level Slug

In [123]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.page_type!="blog"], ["country_code", "slug_root", "event_name"])
#g.sort_values(["country_code", "slug_root", "event_name"])

#filtering where s_count or k_count is >1000
pv = pd.pivot_table(g[(g.s_count>1000) | (g.k_count>1000)], index=["country_code", "slug_root"], values=["k_count","s_count","k_vs_s_%"], columns=["event_name"], fill_value="")

#colour_grouped_table(pv)
#TODO: not showing up the s_count and k_count :(
pv = pv.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [124]:
pv.to_excel("kinesis_vs_segment.xlsx")

In [125]:




#pv[:20].style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, 's-count'])
styled = pv.style.applymap(highlight_cols, subset=pv.columns.get_loc_level('k_vs_s_%', level=1)[0]) #special multi-index on column for colour
#can't get they styling to be happy with colors
#styled.to_excel("kinesis_vs_segment.xlsx", engine='openpyxl') #use special engine for formatting https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

styled

In [126]:
#pv.to_html("kinesis_vs_segment.html")

## By Type of Page

In [127]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "page_type"])
colour_grouped_table(g)

,country_code,page_type,s_count,k_count,is_bot,k_vs_s_%
5,hk,blog,351453,350353,835,-0.3
6,hk,iss,18497,18404,0,-0.5
7,hk,lps,5843,5868,27,0.4
8,hk,shop,160100,157083,224,-1.9
20,sg,blog,1.44012e+06,1.45269e+06,1287,0.9
21,sg,iss,40443,37196,33,-8
22,sg,lps,3074,3042,12,-1
23,sg,shop,833431,815338,1739,-2.2
24,sg,unbounce,7279,7425,7,2


### Type of Page, Just Pageviews

In [128]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.event_name=="PageView") & (merged_df_with_meta.country_code.isin(["sg", "hk"]))], ["country_code", "page_type"])
colour_grouped_table(g)

,country_code,page_type,s_count,k_count,is_bot,k_vs_s_%
5,hk,blog,114750,114636,756,-0.1
6,hk,iss,8732,8541,0,-2.2
7,hk,lps,4938,4897,27,-0.8
8,hk,shop,97167,95609,222,-1.6
20,sg,blog,520731,529474,1184,1.7
21,sg,iss,16817,15270,16,-9.2
22,sg,lps,2794,2714,12,-2.9
23,sg,shop,441204,428896,1686,-2.8
24,sg,unbounce,7279,7425,7,2


### By Device Type and Country for pageviews in HK and SG

In [129]:
merged_df_with_meta.head()


,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code,user_agent,device_family,os_family,os_version,browser_family,browser_version,is_bot
index,,,,,,,,,,,,,,,,,,
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
2,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
3,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0
4,00004699-493e-4238-a2ba-8cc38c829d45,PageView,https://www.moneysmart.hk/zh-hk/credit-cards,2020-01-21,1,1,shop,/zh-hk/credit-cards,/credit-cards,control,hk,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,mobile,iOS,13.3,Mobile Safari,13.0.4,0.0


# Other Issues to Check For
* duplicates
* skipping "https" from the url (observed as a current issue)
* certain browsers having issues
* users creating a lot of duplicate events (and doing the above analysis using sum vs count)